## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Vanimo,PG,-2.6741,141.3028,86.18,68,94,9.95,overcast clouds
1,1,Mount Isa,AU,-20.7333,139.5000,103.77,13,40,5.75,scattered clouds
2,2,Saint-Pierre,RE,-21.3393,55.4781,76.68,57,0,4.61,clear sky
3,3,Hilo,US,19.7297,-155.0900,74.93,86,90,5.75,overcast clouds
4,4,Zhuji,CN,29.7188,120.2423,66.88,58,98,6.67,overcast clouds


In [3]:
gmaps.configure(api_key=g_key)

In [4]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [5]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Vanimo,PG,-2.6741,141.3028,86.18,68,94,9.95,overcast clouds
2,2,Saint-Pierre,RE,-21.3393,55.4781,76.68,57,0,4.61,clear sky
5,5,Libreville,GA,0.3925,9.4537,77.02,100,75,9.22,moderate rain
8,8,Kidal,ML,18.4411,1.4078,80.76,39,16,3.56,few clouds
9,9,Nizwa,OM,22.9333,57.5333,83.17,20,6,0.22,clear sky
...,...,...,...,...,...,...,...,...,...,...
681,681,Kampot,KH,10.6167,104.1833,87.85,70,36,4.50,scattered clouds
682,682,Makakilo City,US,21.3469,-158.0858,79.12,63,1,6.91,clear sky
684,684,Dunmore Town,BS,25.5000,-76.6500,79.75,59,100,15.68,overcast clouds
685,685,Nelson Bay,AU,-32.7167,152.1500,82.76,67,38,7.92,scattered clouds


In [6]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()


City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [7]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.

# No empty rows so not needed, but required code is below and commented out:

# preferred_cities_df.dropna()

In [8]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Vanimo,PG,86.18,overcast clouds,-2.6741,141.3028,
2,Saint-Pierre,RE,76.68,clear sky,-21.3393,55.4781,
5,Libreville,GA,77.02,moderate rain,0.3925,9.4537,
8,Kidal,ML,80.76,few clouds,18.4411,1.4078,
9,Nizwa,OM,83.17,clear sky,22.9333,57.5333,
...,...,...,...,...,...,...,...
681,Kampot,KH,87.85,scattered clouds,10.6167,104.1833,
682,Makakilo City,US,79.12,clear sky,21.3469,-158.0858,
684,Dunmore Town,BS,79.75,overcast clouds,25.5000,-76.6500,
685,Nelson Bay,AU,82.76,scattered clouds,-32.7167,152.1500,


In [9]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key,
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params['location'] = f'{lat},{lng}'
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
        
    except (IndexError):
        print("Hotel not found... skipping.")    
        hotel_df.loc[index, "Hotel Name"] = None
        # Added last line as the default hotel name "" was not showing as null value, easier to drop this way. 

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [10]:
hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Vanimo,PG,86.18,overcast clouds,-2.6741,141.3028,Vahmoneh Lodge Vanimo
2,Saint-Pierre,RE,76.68,clear sky,-21.3393,55.4781,Lindsey Hôtel
5,Libreville,GA,77.02,moderate rain,0.3925,9.4537,Hotel Re-Ndama
8,Kidal,ML,80.76,few clouds,18.4411,1.4078,None
9,Nizwa,OM,83.17,clear sky,22.9333,57.5333,Orient Apartments
...,...,...,...,...,...,...,...
681,Kampot,KH,87.85,scattered clouds,10.6167,104.1833,Two Moons Hotel
682,Makakilo City,US,79.12,clear sky,21.3469,-158.0858,Marriott's Ko Olina Beach Club
684,Dunmore Town,BS,79.75,overcast clouds,25.5000,-76.6500,Coral Sands Hotel
685,Nelson Bay,AU,82.76,scattered clouds,-32.7167,152.1500,Mantra Nelson Bay


In [11]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.dropna(inplace = True)

In [12]:
hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Vanimo,PG,86.18,overcast clouds,-2.6741,141.3028,Vahmoneh Lodge Vanimo
2,Saint-Pierre,RE,76.68,clear sky,-21.3393,55.4781,Lindsey Hôtel
5,Libreville,GA,77.02,moderate rain,0.3925,9.4537,Hotel Re-Ndama
9,Nizwa,OM,83.17,clear sky,22.9333,57.5333,Orient Apartments
10,Mahebourg,MU,76.24,broken clouds,-20.4081,57.7000,Shandrani Beachcomber Resort & Spa
...,...,...,...,...,...,...,...
681,Kampot,KH,87.85,scattered clouds,10.6167,104.1833,Two Moons Hotel
682,Makakilo City,US,79.12,clear sky,21.3469,-158.0858,Marriott's Ko Olina Beach Club
684,Dunmore Town,BS,79.75,overcast clouds,25.5000,-76.6500,Coral Sands Hotel
685,Nelson Bay,AU,82.76,scattered clouds,-32.7167,152.1500,Mantra Nelson Bay


In [13]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [14]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [15]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))